<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Содержание.</i></b></div>

* [Описание данных.](#descr)
    * Общее описание данных, задачи и описательный анализ с графиками
* [Предобработки.](#preproc)
    * Описание процесса предварительной обработки данных
* [Описание модели, метрик и валидации.](#modeling)
    * Описание модели, схем валидации, признаков и метрик
* [Выводы.](#concl)
    * Анализ результатов
* [Дополнения.](#extra)
    * Возможности развития

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Описание данных.</i></b></div> <a name="descr"></a>

Дано $\approx 15000$ собранных с торговой площадки лотов вторичной недвижимости (квартир) в европейском мегаполисе для того, чтобы с помощью методов моделирования и анализа данных определить факторы, значимые для цены этой недвижимости (цены предложения, которая выступает прокси переменной ценой сделки).

> В данной работе мы делаем попытку объяснить структуру цен квартир в Москве в 2020 г. В работе предполагается, что цена квартиры зависит от ее характеристик, таких как размер и расположение. Таким образом, мы следуем концепции «гедонистических» (hedonic) цен, которая впервые была сформулирована в работе (Haas, 1922) и получила развитие в работах (Griliches, 1961) по ценам автомашин и (Chow, 1967) по ценам на компьютеры. В работе (Lancaster, 1966) содержится изложение экономической теории, на которой основан метод гедонистических цен. В работе (Магнус, Пересецкий, 2010), содержится изложение ставшее основой этого абзаца.

* _Также, важно, что мы данные были получены нами в анонимизированном виде третьими лицами, не содержат возможности идентифицировать каких-либо людей и их персональные данные и не распространяются нами в виде, в котором нам их передали._

**Распределение количества комнат* и площади представлено ниже:**
\* После предобработки и чистки пропусков

* В выборке преобладают квартиры из 3 комнат, площадью 100 метров
* Медианное значение площади $75$ метров, 
* $1/10$ доля всех лотов не превышает 35 метров (`Студии и Апартаменты`), а $90\%$ лотов не превышают $150$ метров

<img src="../imgs/Число комнат и площади.png" alt="Цена за метр">

**Распределение цен предложения за единицу площади представлено ниже:**

<img src="../imgs/Цены за метр.png" alt="Цена за метр">

**Распределение цен предложения от площади представлено ниже:**

<img src="../imgs/Цена на метр^2 от площади.png" alt="Цена за метр от площади">

**Также, посмотрим на площадь и цену за метр в зависимости от числа комнат**
* Интересно, что категория `Студии и Апартаменты` занимает наименьшую площадь, но распределение цен квадратного метра в них правее (значения больше), чем, например, в квартирах с `1 или даже 2 комнатами` (вероятно, причина в имеющейся выборке и другим характеристикам этой категории)

Площадь от числа комнат

<img src="../imgs/Площадь и число комнат.png" alt="Площадь от кол-ва комнат">

<img src="../imgs/Площадь и число комнат (точки).png" alt="Площадь от кол-ва комнат (точки)">

Цена от числа комнат

<img src="../imgs/Кол-во комнат и цена на метр (коробки).png" alt="Цена за метр от кол-ва комнат">

<img src="../imgs/Цены и число комнат (точки).png" alt="Цена за метр от кол-ва комнат (точки)">

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Преодобработки.</i></b></div> <a name="preproc"></a>

> * Удалим пропуски которые нельзя восстановить
* Заполним эвристиками (на основе распределений цен по известным) пропуски в поле `Кол-во комнат`:
    * **все** лоты меньше $31$ метров площади отнесем к категории `Студия или Апартаменты` (`СиА`), туда же отнесем недвижимость классифицированную дающим объявление, как `Аппартаменты`)
    * **все** лоты площадью больше $190$ метров отнесем к категории `Многокомнатная или свободная планировка` (`МиСП`)
    * Остальные лоты с пропусками количества комнат заполним на **основании квантилей распределений площадей лотов с известных количеством комнат, с ограничением квантилями лотов большего числа комнат**
* Заменим указание на еще не открывшиеся станции метро (`откроется в X`) в поле с расстоянием до метро на `NaN` (11 лотов)
* Добавим признаки по образцу из статьи \[[ссылка](https://sun9-40.userapi.com/c857628/v857628789/1f0c46/eSwQt6LHzSg.jpg])\]
* Приведём типы (например, категориальные признаки, 'Округ', 'Район', 'Кол-во комнат', 'Метро', 'Тип дома', 'Окна', 'Ремонт'),  к категориями) и наивно закодируем категории (например, _р-н Гольяново_ просто станет $159$, без учета его связей/интеракци с другими признаками)

<img src="../imgs/Матрица корреляции признаков.png" alt="Цена за метр">

**Интересные характеристики лотов**
* Наличие более одного санузла, почти также взаимосвязано с площадью, как и количество комнат. Но попарно их корреляция относительно невелика.
* Число этажей обратно пропорционально возрасту дома. Наверное, это ожидаемо, при учете медианного возраста дома в $20$ лет ($18$ без обработки пропусков), а медианного количества этажей в доме равного $16$ этажам.

| Этажей в доме | 17.0 | 9.0  | 12.0 | 5.0  | 14.0 | 16.0 | 25.0 | 22.0 | 8.0 | 24.0 |
|---------------|------|------|------|------|------|------|------|------|-----|------|
| Число лотов   | 1790 | 1477 | 1089 | 1005 | 925  | 803  | 618  | 570  | 458 | 423  |

* Район более скоррелирован как с абсолютной ценой, так и с ценой за метр квадратный, чем округ
* По не ясной причине, доля куханной площади скореллирована обратно и относительно сильно с площадью. Вероятно, дело в пропусках в данных и в структуре выборке

* `scipy.stats.spearmanr(df['Кол-во комнат'].values, df['Несколько санузлов'].values)`
    * SpearmanrResult(correlation=0.27, pvalue=4.587435724451326e-242)
* `scipy.stats.spearmanr(df['Площадь'].values, df['Несколько санузлов'].values)`
    * SpearmanrResult(correlation=0.51, pvalue=0.0)
* `scipy.stats.spearmanr(df['Площадь'].values, df['Кол-во комнат'].values)`
    * SpearmanrResult(correlation=0.60, pvalue=0.0)
***    
* ```scipy.stats.spearmanr(df['Этажей в доме'].values, df['Возраст дома'].fillna(df.groupby(['Кол-во комнат'])['Возраст дома'].transform('median')))```
    * SpearmanrResult(correlation=-0.5032993608981183, pvalue=0.0)

***
* ```np.corrcoef(df['Площадь'].values, df['Доля куханной площади'].fillna(df.groupby('Кол-во комнат')['Доля куханной площади'].transform('median')))```
    * [[ 1.        , -0.43344788],
       [-0.43344788,  1.        ]]

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Описание модели, метрик и валидации.</i></b></div> <a name="modeling"></a>

> * Мы взяли **алгоритм градиентного бустинга** (имплементация lightgbm) и решали задачу регрессии
* Параметры подбирались интуитивно, с последующей валидацие на отложенной выборке с большей регуляризацией
* Основная модель валидации: отложенная выборка $50\%$, со стратификацией по кол-ву комнат.
    * То есть, мы моделируем ситуацию, когда у нас было $\approx 7000$ лотов, а потом пришло еще 7000 тысяч, с примерным сохранением количества комнат*, и мы хотим узнать, насколько хорошо наша модель может объяснить цену на них.
    * Проблемой такого подхода может быть недостаточное количество наблюдений по категориям чтобы построить хорошую модель на них. Например, всего лотов _категории Z_ меньше $1000$, и в них есть высокая неоднородность.
* С точки зрения метрик качества, в зависимости от степени регуляризации модели и категорий: 
    * $R^2$ лежит в интервале от $0.75$ до $0.9$ (доля объясненной дисперсии) 
    * Относительные ошибки прогнозов лежат в интервале от $0.08$ до $0.3$ (от $\pm 8\%$ для студий до $\pm 30\%$ для неоднородных лотов $190+$ метров с отложенной выборкой в половину всех из них)

**Пример лога перед обучением:**
```
INFO:root:Моделируем: Цена за метр^2.
INFO:root:Не используем: ['Площадь', 'Площадь кухни'].
INFO:root:Используем: ['Округ', 'Район', 'Площадь лог', 'Кол-во комнат', 'Этаж', 'Этажей в доме', 'Первый этаж', 'Последний этаж', 'Метро', 'Расстояние до метро (ед)', 'Расстояние указано пешком', 'Расстояние указано на транспорте', 'Тип дома', 'Указан раздельный санузел', 'Несколько санузлов', 'Балкон указан', 'Окна', 'Окна во двор', 'Ремонт', 'Наличие ремонта', 'Возраст дома', 'Доля жилой площади', 'Доля куханной площади'].
```

<img src="../imgs/Модель Цена за метр Прогнозы.jpg" alt="Цена за метр прогнозы">

<img src="../imgs/importrance_whole_data.png" alt="Цена за метр факторы">

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Выводы.</i></b></div> <a name="concl"></a>

> **Отчет 2: Влияние факторов на цену за метр у лотов**
* Мы видим, что на нашей выборке самыми значимыми факторами влияющими на **цену предложения на метр^2** в 'обобщенной' Московской квартиры на рынке вторичной недвижимости оказываются **район и станция метро**. Причем, наибольший информационный вклад на имеющейся выборке и с текущей моделью вносит именно **район**, хотя незначительно чаще для дополнительного объяснения цены в алгоритме используется **станция метро**.
* Также, именно **районы и станции метро** встречаются в первых деревьях, и даже узлах деревьев бустинга наиболее часто.
* Далее, **по мощности вклада** следуют возраст дома (также, он обратно связан с количеством этажей), вид ремонта (также, он еще и относительно часто встречается), площадь лота (которая, также, оказывает наибольшее влияние на абсолютную цену квартиры) и количество этажей. С другой стороны, **наиболее часто** в узлах наших деревьев далее встречаются возраст дома, площадь и количество этажей в доме. А так как, количество этажей сильно обратно скорелированно с возрастом дома, то есть, наиболее вероятно, что эти два критерия позволяют модели лучше разделить нашу выборку и уточняют зависимости в данных, а не используются в качестве попарной замены.
* Доля куханной площади вносит относительно высокий вклад, и также часто встречается. В похожей, но менее значимой ситуации находятся этаж и тип дома.
* Также, относительно часто встречаются такие факторы, как доля жилой площади, расстояние до метро, количество комнат (связано с площадью), однако их вклад не велик и вероятно они используются для разделение спорных лотов.

> **Дополнительно:** Из интересного, 
* Даже методами описательного анализа можно подвердить вывод из статьи (Магнус, Пересецкий, 2011) о том, что цена за метр в квартирах большей площади возрастает. Также, она возрастает в квартирах с большим числом комнат, и, вероятно, возрастает и в квартирах с большим числом санузлов, потому что наличие более одного санузла сильнее скоррелировано с площадью, чем с количеством комнат
* В зависимости от категории кол-ва комнат лота, значимость признаков может меняться. Однако, ни район, ни станция метро не покидают топ-5

<div style="border-left: solid 6px #d6d6d6; padding: 10px; background-color: #eaeaea">
<b><i>Дополнения.</i></b></div> <a name="extra"></a>

**Расстояние до центра или до экономического центра или иного центра прятижения**
* Для измерения можно больше учитывать соц.дем аудитории площадки и пытаться прогнозировать вероятные точки притяжения. **Например**, Москва-сити, БЦ Белая площадь на Белорусском вокзале, офис Яндекса, правительство Москвы. Если это вообще возможно. В текущей ситуации, предположу, что наилучшая прокси-переменная этого -- район.

**Влияние ветки метро**
* Для измерения следует обработать данные по станциям, которые встречаются в более чем одной ветке. Например, станция Третьяковская по данным с портала открытых данных города Москвы ([ссылка](https://data.mos.ru/classifier/7704786030-stantsii-moskovskogo-metropolitena) расположена одновременно на трех ветках, одна из которых еще строится. Доступная выгрузка из источника данных не дает такой подробной информации.

<div style="border-left: solid 0px #d6d6d6; padding: 10px; background-color: #eaeaea">